Name:

Section:

# Decision Trees Exercise
This exercise will guide you in implementing the Decision Trees. 


## Instructions
* Read each cell and implement the TODOs sequentially. The markdown/text cells also contain instructions which you need to follow to get the whole notebook working.
* Do not change the variable names unless the instructor allows you to.
* Answer all the markdown/text cells with "A: " on them. The answer must strictly consume **one to two lines** only.
* You are expected to search how to some functions work on the Internet or via the docs. 
* There are commented markdown cells that have crumbs. Do not delete them or separate them from the cell originally directly below it.  
* You may add new cells for "scrap work" as long as the crumbs are not separated from the cell below it.
* The notebooks will undergo a "Restart and Run All" command, so make sure that your code is working properly.
* You are expected to understand the data set loading and processing separately from this class.
* You may not reproduce this notebook or share them to anyone.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['figure.figsize'] = (12.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

# Fix the seed of the random number 
# generator so that your results will match ours
np.random.seed(1)

%load_ext autoreload
%autoreload 2

## Decision tree classifier

For this first section, we will create a decision tree to predict the flower species from the iris dataset.

**Dataset:**
The iris data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant. One class is linearly separable from the other 2; the latter are NOT linearly separable from each other. 

Attribute Information:

1. Sepal length in cm
2. Sepal width in cm
3. Petal length in cm
4. Petal width in cm
5. Class (Species):
    - Iris Setosa
    - Iris Versicolour
    - Iris Virginica

We will be using pandas to import our csv data into Python

In [ ]:
import pandas as pd

Load the csv file into a pandas dataframe

In [ ]:
iris = pd.read_csv('iris.csv')
iris

In [ ]:
iris.head()

Extract the feature columns for X, and the label column for y

In [ ]:
# write code here
X = None
y = None

Split the dataset into train and test sets. Set the `random_state` to `0`. 

In [ ]:
# write code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = None

## Building our classification tree
We will be using sklearn's `DecisionTreeClassifier`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Create a classification tree here but do not make any predictions in this cell yet.

In [ ]:
# write code here
dtc = None

From our lecture, we learned that trees can overfit by creating a separate node for every single configuration of feature values possible. Let's see if this is true by running predictions on our training set.

Run predictions on the train set, get the accuracy

In [ ]:
# write code here
predictions_train = None

We will be computing for the accuracy multiple times in this notebook, so let's create a function for this.

`compute_accuracy()` will compute for the accuracy given two vectors of equal length

__Inputs:__
- `predictions`: A numpy array of shape `(N,)` consisting of `N` samples representing the predicted values
- `actual`: A numpy array of shape `(N,)` consisting of `N` samples representing the actual (target) values

__Outputs:__
- `accuracy`: A scalar representing the percentage of elements where `predictions` and `actual` match out of the total number of elements

In [ ]:
def compute_accuracy(predictions, actual):
    # write code here
    return None

In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")

**Sanity Check:** A decision tree without regularization should be able to achieve 100% accuracy on the training set.

If not, you might be seeing a 99.9999% accuracy. But if you look at the predictions, you will see that it got all the train set predicted correctly.

Let's see how our model does with the test set. Run predictions on the test set, and get the accuracy

In [ ]:
# write code here
predictions = None

In [ ]:
print("Testing accuracy: ", compute_accuracy(y_test, predictions),"%")

__Question #1:__ What could possibly contribute to the difference in the train and test error?

<!--crumb;qna;Question: What could possibly contribute to the difference in the train and test error?-->

A: 

## Visualizing the tree

You can also read more about the following code here: https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py 

The code just reconstructs the tree given the model/estimator.

In [ ]:
def describe_tree(clf):
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold
    values = clf.tree_.value

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        # `pop` ensures each node is only visited once
        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        # If the left and right child of a node is not the same we have a split
        # node
        is_split_node = children_left[node_id] != children_right[node_id]
        # If a split node, append left and right children and depth to `stack`
        # so we can loop through them
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    print(
        "The binary tree structure has {n} nodes and has "
        "the following tree structure:\n".format(n=n_nodes)
    )
    for i in range(n_nodes):
        if is_leaves[i]:
            print(
                "{space}node={node} is a leaf node, values: {values}.".format(
                    space=node_depth[i] * "\t", node=i, values=values[i]
                )
            )
        else:
            print(
                "{space}node={node} is a split node: "
                "go to node {left} if X[:, {feature}] <= {threshold} "
                "else to node {right}.".format(
                    space=node_depth[i] * "\t",
                    node=i,
                    left=children_left[i],
                    feature=feature[i],
                    threshold=threshold[i],
                    right=children_right[i],
                )
            )

In [ ]:
describe_tree(dtc)

__Question #2:__ How many nodes are present in the tree above?

<!--crumb;qna;Question: How many nodes are present in the tree above?-->

A: 

__Question #3:__ We have the following features: `sepal_length`, `sepal_width`, `petal_length`, `petal_width`. Which feature does the root node look at?

<!--crumb;qna;Question: We have the following features: sepal_length, sepal_width, petal_length, petal_width. Which feature does the root node look at?-->

A: 

__Question #4:__ If a test data gets categorized to node=12, which among these classes will it fall under: setosa, versicolor, or virginica?

<!--crumb;qna;Question: If a test data gets categorized to node=12, which among these classes will it fall under: setosa, versicolor, or virginica?-->

A: 

In [ ]:
from sklearn import tree

tree.plot_tree(dtc)
plt.show()

In [ ]:
tree.export_graphviz(dtc,out_file='tree.dot')                

To view a graph of the tree, open `tree.dot` and paste its contents in <a src="http://webgraphviz.com/">webgraphviz.com</a>. You'll end up with a similar tree like the one below:

<img src="https://i.imgur.com/E7UJJZk.png" width="300px">

_______

# Decision trees regressor
For regression, we will generate a dummy dataset following a sin curve so we can visualize the results. This will also allow us to visualize how our model does regularization.

## Generating our dataset

In [ ]:
n_samples = 300
np.random.seed(1)
X = np.expand_dims(np.random.uniform(-np.pi,np.pi, n_samples),1)
y = np.sin(2*X) + np.random.randn(n_samples,1)*0.3

num_items = X.shape[0]
randIdx = np.arange(num_items)

Split our dataset. Set the `random_seed` to `42` so your results matches the sanity check below.

In [ ]:
# write code here
X_train, X_test, y_train, y_test = None

print("X_train shape : ", X_train.shape)
print("y_train shape : ", y_train.shape)
print("X_test shape : ", X_test.shape)
print("y_test shape : ", y_test.shape)

We will need our `y` as vectors later on, so let's reshape/squeeze that extra dimension off.

In [ ]:
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

### Visualizing the dataset
To help you visualize our data, it looks like this:

In [ ]:
plt.scatter(X_train,y_train)

## Building our regression tree
Here we will use `DecisionTreeRegressor`. 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

Build our classifier, and make it run predictions on our train data. Use the default hyperparameters for our regressor.

In [ ]:
# write code here
dtr = None


train_predictions = None
train_predictions

Calculate for the mean squared error and the mean absolute error. We will make a function for both of these because will be computing for the `mse` and `mae` multiple times in the notebook.

___

`compute_mse()` will compute for the mean squared error given two vectors of equal length

__Inputs:__
- `predictions`: A numpy array of shape `(N,)` consisting of `N` samples representing the predicted values
- `actual`: A numpy array of shape `(N,)` consisting of `N` samples representing the actual (target) values

__Outputs:__
- `mse`: A scalar representing the mean squared error between `predictions` and `actual`

___

`compute_mae()` will compute for the mean absolute error given two vectors of equal length

__Inputs:__
- `predictions`: A numpy array of shape `(N,)` consisting of `N` samples representing the predicted values
- `actual`: A numpy array of shape `(N,)` consisting of `N` samples representing the actual (target) values

__Outputs:__
- `mae`: A scalar representing the mean absolute error between `predictions` and `actual`

In [ ]:
def compute_mse(predictions, actual):
    # write code here
    return None

def compute_mae(predictions, actual):
    # write code here
    return None

In [ ]:
mse = compute_mse(train_predictions, y_train)
print("Mean Squared Error :", mse)

mae = compute_mae(train_predictions, y_train)
print("Absolute Relative Error :", mae)

## Visualizing our predictions

In [ ]:
plt.scatter(X_train, y_train, color="gray")
plt.scatter(X_train, train_predictions, color="red", marker="x")

__Question #5:__ How can you describe the accuracy of the model on the training data?

<!--crumb;qna;Question: How can you describe the chart above showing y_train vs train_predictions?-->

A: 

In [ ]:
tree.plot_tree(dtr)
plt.show()

Now, make it run predictions on our test data

In [ ]:
# write code here
test_predictions = None

test_predictions

In [ ]:
mse = compute_mse(test_predictions, y_test)
print("Mean Squared Error :", mse)

mae = compute_mae(test_predictions, y_test)
print("Absolute Relative Error :", mae)

**Sanity Check:** You should see values like the following:
```
Mean Squared Error : 0.260963821953
Absolute Relative Error : 0.414856812765
```

Visualize how the our predictions fare against the actual results

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X_test, y_test, label='Ground Truth')
plt.scatter(X_test, test_predictions, label='Predicted')
plt.legend(loc='upper right')

# Regularizing decision trees
In the absence of regularization, the decision tree will memorize the training set and achieve 0% error. While this is good in terms of bias, it may not generalize well to never before seen data (variance problem)

Let's modify our model to include three ways of regularization:
- **Minimum samples split**: If the remaining samples are less than the specifed value then we stop splitting and make it a leaf node
- **Max depth**: Restricts the maximum depth of the trees
- **Minimum impurity decrease**: If the impurity decrease is less than the specified value then we stop splitting and make it a leaf node.
- **Max leaf node**: If the number of leaf nodes is met then we stop splitting.

You will use this test set for the visualization below, so use this as your test set for the loop below. It follows the same sin function as our training data.

In [ ]:
X_test_vis = np.expand_dims(np.linspace(-np.pi,np.pi,300),-1)

In [ ]:
y_test_vis = np.sin(2*X_test_vis) + np.random.randn(n_samples,1)*0.3

In [ ]:
plt.scatter(X_test_vis,y_test_vis, color="gray")

## Regularizing with minimum samples split

Let's apply different minimum samples splits to your model. Let's try it with these values:

In [ ]:
min_samples_split_vals = [2, 4, 6, 10, 15, 20]

To make the following code work, __you have to do the following per iteration__:
1. Create a decision tree regressor following the minimum samples split value for that iteration
1. Fitthe model to your train data and the training labels/outputs
1. Get the trained model to make predictions on `X_test_vis`
1. Plot `X_test_vis` (x-coordinate) relative to the predictions (y-coordinate) made by the model

In [ ]:
# Some plotting stuff
plt_ctr = 1
plt.figure(figsize=(15,12))

for val in min_samples_split_vals:
    # write code here
    dtr = None
    
    
    plt.subplot(3,2,plt_ctr)
    
    #wWrite code here
    predictions = None
    
    plt.scatter(X_test_vis, predictions)
    
    plt.title("Min samples = "+ str(val))
    
    plt_ctr += 1

**Sanity Check:**
Your graph should like the one below:
<img src="https://imgur.com/Wibqyx4.png" width="400px">

<br>


As we increase the number of minimum samples, the predictions for close/nearby x values tend to get the same y-value. 'Minimum samples' refers to the minimum number of instances/samples in a node needed to split it. Increasing the number of samples tells the model to stop splitting early on.

__Question #6:__ For this particular dataset, how many nodes can we expect the final tree to have if we set the minimum sample split to `10,000?`

<!--crumb;qna;Question: For this particular dataset, how many nodes can we expect the final tree to have if we set the minimum sample split to 10,000?-->

A: 

## Regularizing with maximum depth

Let's apply different maximum depths to your model. Let's try it with these values:

In [ ]:
max_depth_vals = [2, 4, 6, 10, 15, 20]

To make the following code work, __you have to do the following per iteration__:
1. Create a decision tree regressor following the maximum depth value for that iteration
1. Fit the model to your train data and the training labels/outputs
1. Get the trained model to make predictions on `X_test_vis`
1. Plot `X_test_vis` (x-coordinate) relative to the predictions (y-coordinate) made by the model

In [ ]:
plt_ctr = 1
plt.figure(figsize=(15,12))
for val in max_depth_vals:
    # write code here
    dtr = None
    
    
    plt.subplot(3,2,plt_ctr)
    
    # write code here
    predictions = None
    
    plt.scatter(X_test_vis, predictions)
    
    plt.title("Max depth = "+ str(val))
    
    plt_ctr += 1

**Sanity Check:** The higher the max depth, the more unique labels you will get. (Inverse of minimum spits)

As we increase the maximum depth, the predictions for close/nearby x values tend to get the different y-values. An increase in maximum depth tells the model it's okay to keep splitting more nodes. Having a small max depth tells the model to stop after it reachers the specified depth/height.

## Regularizing with minimum impurity decrease

Let's apply different minimum impurity decrease to your model. Let's try it with these values:

In [ ]:
min_impurity_vals = [0, 0.0001, 0.001, 0.01, 0.1, 1]

To make the following code work, __you have to do the following per iteration__:
1. Create a decision tree regressor following the minimum impurity decrease value for that iteration
1. Fit the model to your train data and the training labels/outputs
1. Get the trained model to make predictions on `X_test_vis`
1. Plot `X_test_vis` (x-coordinate) relative to the predictions (y-coordinate) made by the model

In [ ]:
plt_ctr = 1
plt.figure(figsize=(15,12))

for val in min_impurity_vals:
    # write code here
    dtr = None
    
    
    plt.subplot(3,2,plt_ctr)
    
    # write code here
    predictions = None
    
    plt.scatter(X_test_vis, predictions)
    
    plt.title("Min impurity decrease= "+ str(val))
    
    plt_ctr += 1

**Sanity Check:** You should see an output with a similar pattern as minimum samples split.

As we increase the minimum impurity decrease, the predictions for close/nearby x values tend to get the same y-value. Having a lower impurity decrease tells the model to reach this level of minimum impurity before a node stop branching out. So **having a lower impurity requires the model to keep on splitting until every node achieves the minimum impurity**. With an imposed high impurity decrease, the model can stop splitting if it does not lower the impurity in the next split.

## Regularizing with max leaf nodes 

Let's apply different max leaf nodes to your model. Let's try it with these values:

In [ ]:
max_leaf_vals = [3, 5, 10, 20, 50, 100]

To make the following code work, __you have to do the following per iteration__:
1. Create a decision tree regressor following the max leaf nodes value for that iteration
1. Fit the model to your train data and the training labels/outputs
1. Get the trained model to make predictions on `X_test_vis`
1. Plot `X_test_vis` (x-coordinate) relative to the predictions (y-coordinate) made by the model

In [ ]:
plt_ctr = 1
plt.figure(figsize=(15,12))

for val in max_leaf_vals:
    # write code here
    dtr = None
    
    
    plt.subplot(3,2,plt_ctr)
    
    # write code here
    predictions = None
    
    plt.scatter(X_test_vis, predictions)
    
    plt.title("Max leaf nodes= "+ str(val))
    
    plt_ctr += 1

**Sanity Check:** You should see an output with a similar pattern as max depth.

As we increase the max leaf nodes, the predictions for close/nearby x values tend to get the different y-values. Why? Increasing the max leaf nodes allow our model to split data into their own value node, while having a small max leaf nodes forces our model to stop splitting early.

# Hyperparameter tuning

Now that we have learned the hyperparameters we can control in our tree model, let's try to find out the best hyperparameters for our model using cross validation. We will also use randomized search to aid in hyperparameter tuning.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

## Tuning a decision tree regressor
Note that we will be using the same sin wave data for the following section. 

We already separated our training data (`X_train`, `y_train`) from our hold-out test set (`X_test`, `y_test`) so we can go straight to modelling.

### Training our model
Let's define our base model/estimator. Since this is a regression task, we should make a `DecisionTreeRegressor`.

In [ ]:
# write code here
dtr = None

In [ ]:
dtr.get_params().keys()

For the following section, we will define our hyperparameters. For now, set the following hyperparameter choices:

__Hyperparameters__:
- minimum impurity decrease could be 0.001, 0.01, 0.05, 0.1, 0.3, 0.5
- max depth could be 5, 10, 20, 30
- minimum samples split could be 2, 4, 6, 10, 15, 20
- max leaf nodes could be 3, 5, 10, 20, 50, 100

In [ ]:
# write code here
hyperparameters = [
    {
        #'<name>': <value>,
    }
]

Create our `RandomizedSearchCV` object, and give it our base estimator and hyperparameters. Set the number of estimators to `50`, and the number of cross validation folds to `5`. Set the `random_state` to `42`.

In [ ]:
# write code here
rsr = None

Train our models on our training data

In [ ]:
# write code here


Find the best hyperparameters found by our randomized search

In [ ]:
# write code here


Now, we can make a new estimator with the best hyperparameters.

In [ ]:
# write code here
dtr = None

Train the estimator on our data:

In [ ]:
# write code here


And, get the predictions and the mean squared error

In [ ]:
# write code here
predictions = None

print("MSE: ", compute_mse(y_train, predictions))
print("MAE: ", compute_mae(y_train, predictions))

__Question #7:__ What is the training mean squared error of our best model?

<!--crumb;qna;Question: What is the training mean squared error of our best model?-->

A: 

__Question #8:__ What is the training mean absolute error of our best model?

<!--crumb;qna;Question: What is the training mean absolute error of our best model?-->

A: 

__Note:__ We could directly use `rsr` to make predictions as it refits to our provided data by default.

**Visualizing how our model performs on the training data**

In [ ]:
plt.scatter(X_train, y_train, color="gray")
plt.scatter(X_train, predictions, color="red")

### Testing phase

Now, let's test our model on our test data. If we tuned our models correctly, we should be able to get a comparable result.

In [ ]:
# write code here
predictions = None

print("MSE: ", compute_mse(y_test, predictions))
print("MAE: ", compute_mae(y_test, predictions))

**Sanity check:** You should get the following results:
```
MSE:  0.14689992394675497
MAE:  0.29875019754298676
```

**Visualizing how our model performs on the test data**

In [ ]:
plt.scatter(X_test, y_test, color="gray")
plt.scatter(X_test, predictions, color="red")

## Tuning a decision tree classifier
We will bring back the iris dataset from before to train our decision tree.

### Preparing our data
We get our `X` and separate it from our target output `y`

In [ ]:
# write code here
X = None
y = None

### Separating out our hold-out test set
We will use the test set later to check how well our model is doing after we perform hyperparameter tuning. For this split, make sure we stratify our data based on the species. Set the `random_state` to `42`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

To know if the stratification worked, the following code should show an equal number of counts per class:

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print("Training data label counts:")
print(np.array([unique, counts]))

__Sanity check:__ You should see the following
```
Training data label counts:
[[ 0  1  2]
 [35 35 35]]
```

In [ ]:
unique, counts = np.unique(y_test, return_counts=True)
print("Test data label counts:")
print(np.array([unique, counts]))

__Sanity check:__ You should see the following
```
Training data label counts:
[[ 0  1  2]
 [15 15 15]]
```

### Training our model
Since the task is framed as a classification problem, we will create a `DecisionTreeClassifier` as our estimator

In [ ]:
# write code here
dtc = None

In [ ]:
dtc.get_params().keys()

For the following code, we will define our hyperparameters. For now, set the following hyperparameter choices:

__Hyperparameters__:
- criterion could either be `gini` or `entropy`
- max depth could be 5, 10, 20, 30
- minimum samples split could be 2, 4, 6, 10, 15, 20
- max leaf nodes could be 3, 5, 10, 20, 50, 100

In [ ]:
# write code here
hyperparameters = [
    {
        #'<name>': <value>,
    }
]

Create our `RandomizedSearchCV` object, and give it our base estimator and hyperparameters. Set the number of estimators to `50`, and the number of cross validation folds to `5`. Set the `random_state` to `42`.

In [ ]:
# write code here
rsc_iris = None

Fit our models to our training data

In [ ]:
# write code here


Let's get the best parameters found in our hyperparameter search

In [ ]:
# write code here


We could also see the results of each model (and their hyperparameters) using the following code:

In [ ]:
rsc_iris.cv_results_

To easily see the results, let's view it as a `DataFrame`

In [ ]:
# this code will help the model not truncate the hyperparameters section
pd.set_option('display.max_colwidth', None)

rsc_results = pd.DataFrame(rsc_iris.cv_results_)
rsc_results

Get the randomized search's best estimator index to quickly get the entry of the best performing model.

In [ ]:
# write code here
best_index = None
best_index

In [ ]:
rsc_results.loc[best_index]

Get the best accuracy achieved by a best model  

In [ ]:
# write code here
best_acc = None

__Question #9:__ What is the accuracy achieved by our best estimator?

<!--crumb;qna;Question: What is the accuracy achieved by our best estimator?-->

A: 

We can also get the `best_estimator_`

In [ ]:
rsc_iris.best_estimator_

### Testing phase
Now that we have our best model, let's see how our best model performs on the test data. 

In [ ]:
# write code here
predictions = None

Finally, compute for the test accuracy

In [ ]:
print("Test accuracy is : ", compute_accuracy(predictions, y_test), "%")

__Question #10:__ What is the test accuracy of our tuned model?

<!--crumb;qna;Question: What is the test accuracy of our tuned model?-->

A: 

# Summary
Decision trees are models that partition your data one feature and one feature threshold/value at a time. This creates a tree that acts like a step-by-step "flowchart" of what to label a new test instance. It is also for this reason that decision trees are non-linear, the end result is a string of decisions across different features. Because of the visualization we get from decision trees, we can thoroughly understand why an instance labelled that way it was by following the "prediction path" the test instance made in the model. (However, this is only true for trees that are shallow. Complex trees are also hard to interpret)

Decision trees are easy to overfit: it can just continue to split/branch off until each instance has its own separate leaf node. We can apply regularization methods like the ones we use above to prevent them from overfitting.

Overfitting, as we have learned, is a sign of a high variance model. In the next lesson, we will learn more about another way of reducing our model's variance (and bias) error with ensemble models.

## <center>fin</center>


<!-- DO NOT MODIFY OR DELETE THIS -->

<sup>made/compiled by daniel stanley tan & courtney anne ngo 🐰 & thomas james tiam-lee</sup> <br>
<sup>for comments, corrections, suggestions, please email:</sup><sup> danieltan07@gmail.com & courtneyngo@gmail.com & thomasjamestiamlee@gmail.com</sup><br>
<sup>please cc your instructor, too</sup>
<!-- DO NOT MODIFY OR DELETE THIS -->